In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier


def do_inner_fold(getModel, inner_cv, X_train_outer, y_train_outer, X_test_outer, lambdas):
        # ---------------------------------------
    # Inner CV for Logistic Regression
    # ---------------------------------------
    
    best_lambda = None
    best_logreg_score = np.inf  # lower is better since we use error rate
    for lam in lambdas:
        val_errors = []
        # Tune on inner folds
        for inner_train_idx, val_idx in inner_cv.split(X_train_outer, y_train_outer):
            X_train_inner, y_train_inner = X_train_outer[inner_train_idx], y_train_outer[inner_train_idx]
            X_val, y_val = X_train_outer[val_idx], y_train_outer[val_idx]
            
            # Note: C = 1/lam
            model = getModel(lam)
            model.fit(X_train_inner, y_train_inner)
            y_val_pred = model.predict(X_val)
            # Classification error = misclassified observations / N_test = 1 - accuracy
            val_error = calculate_errors(y_val, y_val_pred)
            # overfitting error = calculate_errors(y_train_inner, model.predict(X_train_inner)) # degugging purpose
            # print(f"(overfitting check) Validation error for lambda {lam}: {:.4f},  {val_error:.4f}") # degugging purpose
            val_errors.append(val_error)
        mean_val_error = np.mean(val_errors)
        if mean_val_error < best_logreg_score:
            best_logreg_score = mean_val_error
            best_lambda = lam

    # Retrain logistic regression on full outer training set with best lambda
    best_logreg = getModel(best_lambda)
    best_logreg.fit(X_train_outer, y_train_outer)
    y_test_pred_logreg = best_logreg.predict(X_test_outer)
    
    return best_lambda, y_test_pred_logreg

In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

def calculate_errors(y_true, y_pred):
    """
    Calculate classification error.
    :param y_true: True labels
    :param y_pred: Predicted labels
    :return: Classification error (1 - accuracy)
    """
    misclassified = np.sum(y_true != y_pred)
    COUNT = len(y_true)
    inaccuracy = (misclassified / COUNT)
    return inaccuracy

def calculate_baseline_predictions(y_train_outer, y_test_outer):
    # ---------------------------------------
    # Baseline: Predict the majority class
    # ---------------------------------------
    majority_class = np.bincount(y_train_outer).argmax()
    baseline_preds = np.full_like(y_test_outer, majority_class)
    return baseline_preds


In [55]:

from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
def get_parameters_and_target():
    # -------------------------
    # Load and preprocess data
    # -------------------------
    # Change the filename/path as needed
    df = pd.read_excel(".\\datasets\\concrete\\Concrete_Data.xls")

    # Binning the compressive strength into 6 categories
    strength_col = 'Concrete compressive strength(MPa, megapascals) '
    # Use KBinsDiscretizer to create 6 bins based on quantiles
    kbin = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
    df['target'] = kbin.fit_transform(df[[strength_col]]).astype(int)

    # Separate features and target; drop the original target column
    X = df.drop(columns=[strength_col, 'target']).values
    y = df['target'].values

    # Normalize features: each column gets zero mean and unit variance.
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return X, y, df.drop(columns=['target']).columns[:-1]

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

def do_two_layer_cv(X, y):

    # -------------------------------
    # Hyperparameter grids to search
    # -------------------------------
    # For logistic regression, we use lambda (λ) values and note that scikit-learn's C = 1/λ.
    lambdas = np.logspace(-10, 10, 20)  # 10 values between 10^-4 and 10^2

    # For ANN, use the number of hidden units as the complexity controlling parameter.
    hidden_units_list = [1]

    # -------------------------------------------
    # Outer cross-validation: same splits for all
    # -------------------------------------------
    K_outer = 10  # outer folds
    K_inner = 10  # inner folds for hyperparameter tuning

    outer_cv = StratifiedKFold(n_splits=K_outer, shuffle=True, random_state=42)

    # This list will store: (Fold, best_lambda, logistic_error, best_h, ann_error, baseline_error)
    results = []
    target_predictions = np.empty((len(y), 4), dtype=int)

    print(f"{'Fold':<5}{'Best λ':<10}{'LogReg Err':<12}{'Best h':<10}{'ANN Err':<12}{'Baseline Err':<15}")
    for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X, y), 1):
        # Outer training and test sets
        X_train_outer, y_train_outer = X[train_idx], y[train_idx]
        X_test_outer, y_test_outer = X[test_idx], y[test_idx]
        
        inner_cv = StratifiedKFold(n_splits=K_inner, shuffle=True, random_state=fold)

        getLogisticRegModel = lambda lam: LogisticRegression(C=1/lam, penalty='l2', solver='liblinear', max_iter=1000)
        best_lambda, y_test_pred_logreg = do_inner_fold(getLogisticRegModel, inner_cv, X_train_outer, y_train_outer, X_test_outer, lambdas)
        logreg_error = calculate_errors(y_test_outer, y_test_pred_logreg)
        
        getAnnModel = lambda h: MLPClassifier(hidden_layer_sizes=(h,), max_iter=3000, n_iter_no_change=10)
        best_h, y_test_pred_ann = do_inner_fold(getAnnModel, inner_cv, X_train_outer, y_train_outer, X_test_outer, hidden_units_list)
        ann_error = calculate_errors(y_test_outer, y_test_pred_ann)
        
        baseline_preds = calculate_baseline_predictions(y_train_outer, y_test_outer)
        baseline_error = calculate_errors(y_test_outer, baseline_preds)
        
        # Store predictions for the current fold
        target_predictions[test_idx, 0] = y_test_outer
        target_predictions[test_idx, 1] = y_test_pred_ann
        target_predictions[test_idx, 2] = y_test_pred_logreg
        target_predictions[test_idx, 3] = baseline_preds
        
        results.append((fold, best_lambda, logreg_error, best_h, ann_error, baseline_error))
        print(f"{fold:<5}{best_lambda:<10.4f}{logreg_error:<12.4f}{best_h!s:<10}{ann_error:<12.4f}{baseline_error:<15.4f}")

    # Save results to CSV (optional)
    df_results = pd.DataFrame(results, columns=['Fold', 'Best Lambda', 'LogReg Error', 
                                                'Best Hidden Units', 'ANN Error', 'Baseline Error'])
    df_results.to_csv("combined_model_errors.csv", index=False)
    
    df_target_predictions = pd.DataFrame(target_predictions, columns=['original', 'ann_pred', 'logistic_reg_pred', 'baseline_pred'])
    
    return df_results, df_target_predictions


In [46]:
X,y, column_names = get_parameters_and_target()
df_results, df_target_predictions = do_two_layer_cv(X, y)

Fold Best λ    LogReg Err  Best h    ANN Err     Baseline Err   
1    0.0264    0.2718      1         0.3010      0.6699         
2    0.0000    0.2427      1         0.2913      0.6699         
3    0.0000    0.3301      1         0.2816      0.6699         
4    0.0000    0.3495      1         0.6699      0.6699         
5    0.0000    0.3301      1         0.2524      0.6699         
6    0.0000    0.3495      1         0.3398      0.6699         
7    0.2976    0.2330      1         0.2427      0.6602         
8    0.0000    0.3107      1         0.2913      0.6602         
9    0.0000    0.3786      1         0.3398      0.6602         
10   0.0000    0.2718      1         0.2718      0.6602         


In [9]:
import numpy as np
from scipy.stats import binom, beta

def calculate_p_value(y_true, y_pred_1, y_pred_2):    
    # Determine correctness for each classifier:
    correct_A = (y_true == y_pred_1)
    correct_B = (y_true == y_pred_2)

    # Compute discordant counts:
    # n12: A correct, B wrong
    n12 = np.sum(correct_A & (~correct_B))
    # n21: A wrong, B correct
    n21 = np.sum((~correct_A) & correct_B)

    # Total number of discordant pairs:
    N = n12 + n21

    print("n12 (A correct, B wrong):", n12)
    print("n21 (A wrong, B correct):", n21)
    print("Total discordant pairs, N:", N)

    # Check that we have enough discordant pairs to compute a meaningful interval.
    if N < 5:
        print("Warning: n12+n21 < 5; confidence intervals may be unreliable.")

    # 1. Estimate the difference in proportion:
    # θ̂ = (n12 - n21) / (n12 + n21)
    theta_hat = (n12 - n21) / N
    print("Estimated difference in discordant proportions, θ̂ =", theta_hat)

    # 2. Compute the p-value using the binomial distribution.
    # Under the null hypothesis that both outcomes are equally likely (p = 0.5), 
    # we let m = min(n12, n21) and compute:
    # p_value = 2 * P(X <= m) for X ~ Binom(N, 0.5)
    m = min(n12, n21)
    p_value = 2 * binom.cdf(m, N, 0.5)
    # Ensure p_value does not exceed 1.
    p_value = min(p_value, 1.0)
    print("p-value =", p_value)
    return n12, n21, N, theta_hat, p_value

def do_test(y_true, y_pred_1, y_pred_2):
    # Calculate core statistics and the p-value.
    n12, n21, N, theta_hat, p_value = calculate_p_value(y_true, y_pred_1, y_pred_2)
    
    # For the confidence interval, we focus on the proportion of discordant pairs:
    # p = n12 / (n12 + n21)
    # Then our difference estimate is: θ = 2p - 1.
    # We use a Beta distribution to compute the confidence interval on p.
    # Beta parameters: f = n12 + 1, g = n21 + 1.
    f = n12 + 1
    g = n21 + 1
    alpha = 0.05  # for a 95% confidence interval
    
    # Compute the lower and upper quantiles from the Beta distribution for p.
    p_lower = beta.ppf(alpha / 2, f, g)
    p_upper = beta.ppf(1 - alpha / 2, f, g)
    
    # Transform the confidence limits for p into the confidence limits for θ.
    theta_lower = 2 * p_lower - 1
    theta_upper = 2 * p_upper - 1
    
    print("95% Confidence interval for θ: [{:.4f}, {:.4f}]".format(theta_lower, theta_upper))
    
    # Interpretation: if the interval includes 0, then the difference is not statistically significant.
    if p_value < alpha:
        print("The difference between classifiers is statistically significant.")
    else:
        print("There is no statistically significant difference between the classifiers.")

# Example usage:
# Suppose these are your test-set results:
# y_true: true labels (binary or multi-class; here correctness is determined by comparison)
# y_pred_A: predictions from classifier A
# y_pred_B: predictions from classifier B

# Here, it's assumed that you already have a DataFrame (e.g., df_target_predictions)
# that includes the columns 'original', 'ann_pred', 'logistic_reg_pred', and 'baseline_pred'.

y_true = df_target_predictions['original'].values      # True labels
y_pred_A = df_target_predictions['ann_pred'].values       # Predictions from Model 1 (e.g., ANN)
y_pred_B = df_target_predictions['logistic_reg_pred'].values  # Predictions from Model 2 (e.g., Logistic Regression)
y_pred_C = df_target_predictions['baseline_pred'].values   # Predictions from Model 3 (e.g., Baseline)

print("Comparison: Model 1 vs. Model 2")
do_test(y_true, y_pred_A, y_pred_B)
print("\nComparison: Model 1 vs. Baseline")
do_test(y_true, y_pred_A, y_pred_C)
print("\nComparison: Model 2 vs. Baseline")
do_test(y_true, y_pred_B, y_pred_C)


Comparison: Model 1 vs. Model 2
n12 (A correct, B wrong): 100
n21 (A wrong, B correct): 65
Total discordant pairs, N: 165
Estimated difference in discordant proportions, θ̂ = 0.21212121212121213
p-value = 0.007929770979502983
95% Confidence interval for θ: [0.0595, 0.3549]
The difference between classifiers is statistically significant.

Comparison: Model 1 vs. Baseline
n12 (A correct, B wrong): 456
n21 (A wrong, B correct): 51
Total discordant pairs, N: 507
Estimated difference in discordant proportions, θ̂ = 0.7988165680473372
p-value = 2.314629094731244e-82
95% Confidence interval for θ: [0.7402, 0.8452]
The difference between classifiers is statistically significant.

Comparison: Model 2 vs. Baseline
n12 (A correct, B wrong): 411
n21 (A wrong, B correct): 41
Total discordant pairs, N: 452
Estimated difference in discordant proportions, θ̂ = 0.8185840707964602
p-value = 6.376403079492849e-78
95% Confidence interval for θ: [0.7584, 0.8647]
The difference between classifiers is statis

In [15]:
from sklearn.model_selection import train_test_split
def train_final_best_logreg_model(X, y, df_results: pd.DataFrame):
    # Retrain the best logistic regression model on the entire dataset
    # Split the entire dataset into training and testing sets
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, test_size=0.2)

    # Retrain the best logistic regression model on the full training set
    best_lambda = df_results['Best Lambda'].median()  # Get the best lambda from the results
    best_logreg_full = LogisticRegression(C=1/best_lambda, penalty='l2', solver='liblinear', max_iter=1000)
    best_logreg_full.fit(X_train_full, y_train_full)
    y_test_pred_logreg_full = best_logreg_full.predict(X_test_full)
    logreg_error_full = calculate_errors(y_test_full, y_test_pred_logreg_full)

    # Print the results
    print(f"Logistic Regression Error (Full Dataset): {logreg_error_full:.4f}")

train_final_best_logreg_model(X, y, df_results)

Logistic Regression Error (Full Dataset): 0.3058


In [74]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.base import clone
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.contingency_tables import mcnemar
from scipy.stats import mode

def mcnemar_pvalue(y_true, preds_a, preds_b):
    """
    Computes the McNemar test p-value between two sets of predictions.
    
    Constructs a 2x2 contingency table based on whether each prediction is correct:
    
                        Candidate Correct   Candidate Incorrect
    Baseline Correct          a                      b
    Baseline Incorrect        c                      d
    
    The test then focuses on the off-diagonal cells (b and c).
    """
    # Determine which predictions are correct.
    correct_a = (y_true == preds_a)
    correct_b = (y_true == preds_b)
    
    a = np.sum(correct_a & correct_b)     # Both models correct.
    b = np.sum(correct_a & ~correct_b)    # Baseline correct, candidate wrong.
    c = np.sum(~correct_a & correct_b)    # Baseline wrong, candidate correct.
    d = np.sum(~correct_a & ~correct_b)   # Both models wrong.

    table = np.array([[a, b],
                      [c, d]])
    
    result = mcnemar(table, exact=True)
    return result.pvalue

def forward_feature_selection(model_class, X, y, alpha=0.05, cv_folds=10, random_state=42):
    """
    Performs forward feature selection with nested (outer) cross-validation and uses a 
    McNemar-test criterion to determine whether adding a candidate feature results 
    in statistically significant change (i.e. improvement) in the model predictions.
    
    For the baseline (when no feature is selected), an intercept-only model is built 
    using a dummy constant column. A grid search is performed over candidate values 
    of the hyperparameter 'C' (assumed for LogisticRegression) to pick the intercept-only 
    model that yields the lowest error (highest accuracy).
    
    Parameters:
        model_class  : Uninitialized model class (e.g., LogisticRegression).
        X            : DataFrame with all potential features.
        y            : Series or array of target labels.
        alpha        : Significance threshold for McNemar's test (default 0.05).
        cv_folds     : Number of folds for cross-validation.
        random_state : Random seed for reproducibility.
    
    Returns:
        final_model       : The final model trained on all data with the selected features.
        selected_features : List of the selected feature names.
    """
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
    
    selected_features = []  # Initially, no features are selected.
    all_features = list(X.columns)
    
    # Candidate grid for the baseline intercept-only model (using hyperparameter 'C')
    candidate_cs = [0.001, 0.01, 0.1, 1, 10]
    
    iteration = 1
    while True:
        print(f"\nIteration {iteration}")
        print("Current selected features:", selected_features)
        
        # === Baseline predictions ===
        if len(selected_features) == 0:
            # Create a DataFrame with a constant column for an intercept-only model.
            X_baseline = pd.DataFrame({'const': np.ones(len(X))}, index=X.index)
            best_acc = -np.inf
            best_baseline_preds = None
            best_c = None
            
            for c in candidate_cs:
                model = clone(model_class())
                model.set_params(C=c)
                # Obtain out-of-sample predictions using cross-validation.
                preds = cross_val_predict(model, X_baseline, y, cv=cv, method='predict')
                acc = accuracy_score(y, preds)
                print(f"  Baseline candidate with C={c}: Accuracy = {acc:.4f}")
                if acc > best_acc:
                    best_acc = acc
                    best_baseline_preds = preds
                    best_c = c
                    
            print("Baseline (intercept-only) selected with C =", best_c, "and accuracy =", best_acc)
            baseline_preds = best_baseline_preds
        else:
            # When features have been selected, train a baseline model on them.
            baseline_model = clone(model_class())
            baseline_preds = cross_val_predict(baseline_model, X[selected_features], y, cv=cv, method='predict')
        
        # === Evaluate each candidate feature (those not yet selected) ===
        candidate_pvals = {}
        remaining_features = [f for f in all_features if f not in selected_features]
        for feature in remaining_features:
            candidate_features = selected_features + [feature]
            candidate_model = clone(model_class())
            candidate_preds = cross_val_predict(candidate_model, X[candidate_features], y, cv=cv, method='predict')
            p_val = mcnemar_pvalue(y, baseline_preds, candidate_preds)
            candidate_pvals[feature] = p_val
            print(f"  Considering adding '{feature}': McNemar p-value = {p_val:.4f}")
        
        # Among candidates, only those with p-value below alpha (i.e. statistically significant change)
        significant_candidates = {f: p for f, p in candidate_pvals.items() if p < alpha}
        
        if significant_candidates:
            # Select the candidate with the smallest p-value.
            best_feature = min(significant_candidates, key=significant_candidates.get)
            best_pval = significant_candidates[best_feature]
            print(f"--> Adding feature '{best_feature}' with p-value = {best_pval:.4f}")
            selected_features.append(best_feature)
            iteration += 1
        else:
            print("No candidate feature yields a p-value below alpha; stopping selection.")
            break
    
    print("\nFinal selected features:", selected_features)
    
    # Train the final model on the entire dataset using the selected features.
    final_model = clone(model_class())
    if len(selected_features) > 0:
        final_model.fit(X[selected_features], y)
    else:
        # If no features were selected, train the intercept-only model.
        X_baseline = pd.DataFrame({'const': np.ones(len(X))}, index=X.index)
        final_model.fit(X_baseline, y)
    return final_model, selected_features

# =============================================================================
# Example usage:
# Suppose you have a pandas DataFrame 'X' with all potential predictors and a Series 'y'
# containing the target labels.
#
# For instance:
#   import pandas as pd
#   X = pd.read_csv("your_features.csv")
#   y = pd.read_csv("your_target.csv").squeeze()
#
# Then run forward selection with:
#
#   final_model, selected_features = forward_feature_selection(
#       model_class=LogisticRegression,
#       X=X,
#       y=y,
#       alpha=0.05,
#       cv_folds=5,
#       random_state=42
#   )
#
# You can then evaluate 'final_model' on a hold-out set or via additional cross-validation.
# =============================================================================

X_df = pd.DataFrame(X, columns=column_names)

final_model, selected_features = forward_feature_selection(
    model_class=LogisticRegression,
    X=X_df,
    y=y,
    alpha=0.05,
    cv_folds=10,
    random_state=42
)


Iteration 1
Current selected features: []
  Baseline candidate with C=0.001: Accuracy = 0.3340
  Baseline candidate with C=0.01: Accuracy = 0.3340
  Baseline candidate with C=0.1: Accuracy = 0.3340
  Baseline candidate with C=1: Accuracy = 0.3340
  Baseline candidate with C=10: Accuracy = 0.3340
Baseline (intercept-only) selected with C = 0.001 and accuracy = 0.3339805825242718
  Considering adding 'Cement (component 1)(kg in a m^3 mixture)': McNemar p-value = 0.0000
  Considering adding 'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': McNemar p-value = 0.0192
  Considering adding 'Fly Ash (component 3)(kg in a m^3 mixture)': McNemar p-value = 0.2333
  Considering adding 'Water  (component 4)(kg in a m^3 mixture)': McNemar p-value = 0.0000
  Considering adding 'Superplasticizer (component 5)(kg in a m^3 mixture)': McNemar p-value = 0.0000
  Considering adding 'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': McNemar p-value = 0.1360
  Considering adding 'Fine Aggregate (

In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_rel

def paired_ttest_pvalue(y_true, preds_baseline, preds_candidate):
    """
    Computes a one-tailed paired t-test p-value comparing the errors of two models.
    Here we test whether the baseline errors (residuals) are significantly larger than
    the candidate model’s errors.
    
    Parameters:
      y_true         : True target values.
      preds_baseline : Predictions from the baseline model.
      preds_candidate: Predictions from the candidate model.
      
    Returns:
      p-value from a one-tailed paired t-test.
    """
    # Compute residuals: error = y_true - prediction.
    errors_baseline = np.array(y_true) - np.array(preds_baseline)
    errors_candidate = np.array(y_true) - np.array(preds_candidate)
    
    # Perform a paired t-test on the errors.
    t_stat, p_val_two_tailed = ttest_rel(errors_baseline, errors_candidate)
    
    # We expect the candidate model to be better, which means lower error.
    # Hence, if t_stat > 0, then baseline error is on average higher than candidate error.
    # For a one-tailed test, if t_stat > 0, p_val = half the two-tailed p-value.
    if t_stat > 0:
        p_val = p_val_two_tailed / 2
    else:
        # If t_stat is not positive, the candidate is not improved.
        p_val = 1 - p_val_two_tailed / 2
    return p_val

def forward_feature_selection(model_class, X, y, alpha=0.05, cv_folds=10, random_state=42):
    """
    Performs forward feature selection with nested (outer) cross-validation and uses a 
    McNemar-test criterion to determine whether adding a candidate feature results 
    in statistically significant change (i.e. improvement) in the model predictions.
    
    For the baseline (when no feature is selected), an intercept-only model is built 
    using a dummy constant column. A grid search is performed over candidate values 
    of the hyperparameter 'C' (assumed for LogisticRegression) to pick the intercept-only 
    model that yields the lowest error (highest accuracy).
    
    Parameters:
        model_class  : Uninitialized model class (e.g., LogisticRegression).
        X            : DataFrame with all potential features.
        y            : Series or array of target labels.
        alpha        : Significance threshold for McNemar's test (default 0.05).
        cv_folds     : Number of folds for cross-validation.
        random_state : Random seed for reproducibility.
    
    Returns:
        final_model       : The final model trained on all data with the selected features.
        selected_features : List of the selected feature names.
    """
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
    
    selected_features = []  # Initially, no features are selected.
    all_features = list(X.columns)
    
    # Candidate grid for the baseline intercept-only model (using hyperparameter 'C')
    candidate_cs = [0.001, 0.01, 0.1, 1, 10]
    
    iteration = 1
    while True:
        print(f"\nIteration {iteration}")
        print("Current selected features:", selected_features)
        
        # === Baseline predictions ===
        if len(selected_features) == 0:
            # Create a DataFrame with a constant column (intercept-only model).
            X_baseline = pd.DataFrame({'const': np.ones(len(X))}, index=X.index)
            baseline_model = clone(model_class())
            # Use cross_val_predict to obtain out-of-sample predictions.
            baseline_preds = cross_val_predict(baseline_model, X_baseline, y, cv=cv, method='predict')
            # Optionally, print an error metric.
            baseline_mse = mean_squared_error(y, baseline_preds)
            print(f"Baseline (intercept-only) MSE = {baseline_mse:.4f}")
        else:
            # When features have been selected, train a baseline model on them.
            baseline_model = clone(model_class())
            baseline_preds = cross_val_predict(baseline_model, X[selected_features], y, cv=cv, method='predict')
        
        # === Evaluate each candidate feature (those not yet selected) ===
        candidate_pvals = {}
        remaining_features = [f for f in all_features if f not in selected_features]
        for feature in remaining_features:
            candidate_features = selected_features + [feature]
            candidate_model = clone(model_class())
            candidate_preds = cross_val_predict(candidate_model, X[candidate_features], y, cv=cv, method='predict')
            p_val = paired_ttest_pvalue(y, baseline_preds, candidate_preds)
            candidate_pvals[feature] = p_val
            print(f"  Considering adding '{feature}': McNemar p-value = {p_val:.4f}")
        
        # Among candidates, only those with p-value below alpha (i.e. statistically significant change)
        significant_candidates = {f: p for f, p in candidate_pvals.items() if p < alpha}
        
        if significant_candidates:
            # Select the candidate with the smallest p-value.
            best_feature = min(significant_candidates, key=significant_candidates.get)
            best_pval = significant_candidates[best_feature]
            print(f"--> Adding feature '{best_feature}' with p-value = {best_pval:.4f}")
            selected_features.append(best_feature)
            iteration += 1
        else:
            print("No candidate feature yields a p-value below alpha; stopping selection.")
            break
    
    print("\nFinal selected features:", selected_features)
    
    # Train the final model on the entire dataset using the selected features.
    final_model = clone(model_class())
    if len(selected_features) > 0:
        final_model.fit(X[selected_features], y)
    else:
        # If no features were selected, train the intercept-only model.
        X_baseline = pd.DataFrame({'const': np.ones(len(X))}, index=X.index)
        final_model.fit(X_baseline, y)
    return final_model, selected_features
# =============================================================================
# Example usage:
# Suppose you have a pandas DataFrame 'X' with your predictor variables and a Series 'y'
# with your continuous target variable.
#
# For example:
#   import pandas as pd
#   X = pd.read_csv("your_features.csv")
#   y = pd.read_csv("your_target.csv").squeeze()
#
# Then run forward selection with:
#
#   final_model, selected_features = forward_feature_selection(
#       model_class=LinearRegression,
#       X=X,
#       y=y,
#       alpha=0.05,
#       cv_folds=10,
#       random_state=42
#   )
#
# You can then evaluate 'final_model' on a hold-out set or via additional cross-validation.
# =============================================================================

# Example data (for demonstration purposes, replace with your own data):
# Assume X is a 2D array or DataFrame and y is an array-like of continuous values.
# Here we make some dummy data:

df = pd.read_excel(".\\datasets\\concrete\\Concrete_Data.xls")
strength_col = 'Concrete compressive strength(MPa, megapascals) '
X_linear = df.drop(columns=[strength_col])
y_linear = df[strength_col]
scaler = StandardScaler()
X_linear = pd.DataFrame(scaler.fit_transform(X_linear), columns=X_linear.columns)

final_model, selected_features = forward_feature_selection(
    model_class=LinearRegression,
    X=X_linear,
    y=y_linear,
    alpha=0.05,
    cv_folds=10,
    random_state=42
)



Iteration 1
Current selected features: []


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [89]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.stats import ttest_rel

# Set up cross-validation
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Get predictions from constant model
X_baseline = pd.DataFrame({'const': np.ones(len(X_linear))}, index=X_linear.index)
constant_model = clone(LinearRegression())
const_preds = cross_val_predict(constant_model, X_baseline, y_linear, cv=cv, method='predict')
const_mse = mean_squared_error(y_linear, const_preds)

# Get predictions from the full linear regression model
X_age = X_linear[['Age (day)']]
full_linear_model = LinearRegression()
linear_preds = cross_val_predict(full_linear_model, X_age, y_linear, cv=cv)
linear_mse = mean_squared_error(y_linear, linear_preds)

# Calculate residuals
const_residuals = y_linear - const_preds
linear_residuals = y_linear - linear_preds

# Perform paired t-test on squared residuals (errors)
const_errors = const_residuals**2
linear_errors = linear_residuals**2
t_stat, p_value = ttest_rel(const_errors, linear_errors)

# For a one-tailed test (linear model should be better = smaller errors)
if t_stat > 0:  # If constant errors are larger
    one_tailed_p = p_value / 2
else:
    one_tailed_p = 1 - p_value / 2

print(f"Constant model MSE: {const_mse:.2f}")
print(f"Linear regression MSE: {linear_mse:.2f}")
print(f"Improvement: {const_mse - linear_mse:.2f} ({(1 - linear_mse/const_mse) * 100:.2f}%)")
print(f"Paired t-test: t = {t_stat:.4f}, p-value (one-tailed) = {one_tailed_p:.10f}")

if one_tailed_p < 0.05:
    print("The linear regression model is significantly better than the constant model.")
else:
    print("There is no significant evidence that the linear regression model is better.")

Constant model MSE: 279.13
Linear regression MSE: 249.55
Improvement: 29.58 (10.60%)
Paired t-test: t = 7.0368, p-value (one-tailed) = 0.0000000000
The linear regression model is significantly better than the constant model.
